# Configuration Notebook
Useful for debugging configurations and viewing configuration details.

## Setup

In [ ]:
import sys, os
modules_path = os.path.join('..', 'src')
if modules_path not in sys.path: sys.path.insert(0, modules_path)

from pprint import pformat, pp
from IPython import display

from forgather.config import (
    load_config,
    ConfigEnvironment,
    pconfig
)

from aiws.config import base_preprocessor_globals, MetaConfig
from aiws.training_loop import TrainingScriptConfig
import aiws.notebooks as nb

# Set project and configuration
project_directory = "tiny_models"
config_template = "tiny.yaml"

## Project
Set 'project_directory' to the path of the project directory.

In [ ]:
assert os.path.exists(project_directory), "Project directory does not exist."
nb.show_project_readme(project_directory)
meta = MetaConfig(project_directory)
nb.display_meta(meta, "### Meta Config\n")
nb.list_templates(meta.find_templates(meta.config_prefix), "### Available Configurations\n")

## List Available Templates

In [ ]:
def list_templates(prefix):
    nb.list_templates(meta.find_templates(prefix), "### Templates\n")
list_templates('')

## Configuration
Set 'config_template' to the name of a configuration template in the project.

In [ ]:
config_template_path

In [ ]:
# Create configuration envrionment
environment = ConfigEnvironment(
    searchpath=meta.searchpath,
    globals = base_preprocessor_globals() | dict(project_directory=project_directory)
)

config_template_path = os.path.join(meta.config_prefix, config_template)
nb.display_preprocessed_template(environment, config_template_path, title="### Preprocessed Configuration\n")
nb.display_referenced_templates_tree(environment, config_template_path, "### Included Templates\n")
loaded_config = environment.load(config_template_path)
nb.display_referenced_source_list(loaded_config.config, "### Included Sources\n")
display.display(display.Markdown("### Loaded Configuration\n"))
pconfig(loaded_config.config)

## Materialized Configuration

Instantiate the configuration from the definition.

In [ ]:
from aiws.training_loop import TrainingScriptConfig

loaded_config = environment.load(config_template_path)
config = TrainingScriptConfig(**loaded_config.config)

# Materialize the trainer
# This config format expects the pre-processed config to be injected, as this
# can be used to log the configuration.
config.trainer = config.trainer(pp_config=loaded_config.pp_config)
pconfig(config)

### Test Configuration

Run the trainer in the notebook.
Note: Running directly from the notebook may have issues.

For a more robust approach, see: [train.ipynb](train.ipynb)

In [ ]:
config.trainer.train()

### Cleanup
Note: These will show the target directory and ask for confirmation before proceeding.

#### Delete All

In [ ]:
nb.delete_dir(config.models_dir, "Delete all models in project")

#### Delete Configuration Output Directory
This will delete the model and logs for the current configuration.

In [ ]:
nb.delete_dir(config.output_dir, "Delete output directory")